In [1]:
import numpy as np
from keras.datasets import mnist
from keras.models import Sequential 
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import to_categorical
from skimage.transform import resize
import h5py
import os
import cv2


### Resize as 42x42(Camera like), make it between 0 and 127(Divide each pixel by 2), so that it can be represented in 7 bits

In [2]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train_resized = np.zeros((x_train.shape[0], 42, 42), dtype=np.uint8)
for i in range(x_train.shape[0]):
    x_train_resized[i] = cv2.resize(x_train[i], (42, 42))

x_test_resized = np.zeros((x_test.shape[0], 42, 42), dtype=np.uint8)
for i in range(x_test.shape[0]):
    x_test_resized[i] = cv2.resize(x_test[i], (42, 42))

x_train_resized = x_train_resized.reshape(-1, 42, 42, 1)
x_test_resized = x_test_resized.reshape(-1, 42, 42, 1)

# Divide by 2
for i in range(x_train.shape[0]):
    x_train_resized[i] = x_train_resized[i]/2   

for i in range(x_test.shape[0]):
    x_test_resized[i] = x_test_resized[i]/2

x_test_max = np.max(x_test_resized)
print(x_test_max)
print(x_test_resized.shape)

# One-hot encode labels
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


127
(10000, 42, 42, 1)


### Convert into bits, and then convert again in Floating point but using our Fixed point Representation: [-8, 4, 2, 1, 0.5, 0.25, 0.125, 0.0625] So, 127 -> 7.95

In [3]:
def convert_to_binary(images):
    binary_images = []

    for img in images:
        binary_img = []

        for pixel_value in np.nditer(img):
            binary_str = format(pixel_value, '08b')
            binary_img.append(binary_str)

        binary_images.append(binary_img)

    return binary_images

# Convert x_test_resized and x_train_resized to binary
binary_x_test_resized = convert_to_binary(x_test_resized)
binary_x_train_resized = convert_to_binary(x_train_resized)

In [4]:
def binary_to_float(binary_str):
    weights = [-8, 4, 2, 1, 0.5, 0.25, 0.125, 0.0625]
    
    result = 0
    
    for i, bit in enumerate(binary_str):
        result += int(bit) * weights[i]
    
    return result

def convert_to_float(binary_images):
    float_images = []

    for img in binary_images:
        float_img = []

        for binary_str in img:
            float_value = binary_to_float(binary_str)
            float_img.append(float_value)
        
        float_img = np.array(float_img).reshape((42, 42))
        float_images.append(float_img)

    return np.array(float_images)

# Convert binary_x_test_resized and binary_x_train_resized to floating point values as numpy arrays
FP_x_test_resized = convert_to_float(binary_x_test_resized)
FP_x_train_resized = convert_to_float(binary_x_train_resized)

FP_x_train_resized = FP_x_train_resized.reshape(-1, 42, 42, 1)
FP_x_test_resized = FP_x_test_resized.reshape(-1, 42, 42, 1)

### Model Training (Not needed if you have the model_weights_FP.h5 file)

In [5]:
# import numpy as np
# from keras.models import Sequential
# from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
# from keras.callbacks import ModelCheckpoint

# model = Sequential()
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Flatten())
# model.add(Dropout(0.5))
# model.add(Dense(10, activation='softmax'))

# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# model.fit(FP_x_train_resized, y_train, batch_size=50, epochs=15, validation_data=(FP_x_test_resized, y_test))

# model.save('model_weights_FP.h5')


### Generating txt file(Required for C) from h5

In [10]:
# from keras.models import load_model

# model = load_model('model_weights_FP.h5')

# # Get the weights for each layer
# weights = model.get_weights()

# # Save the weights to a text file
# with open('model_weights_FP.txt', 'w') as file:
#     for layer_weights in weights:
#         for weight in layer_weights.flatten():
#             file.write(str(weight) + '\n')

# print("Model weights saved to model_weights_FP.txt")

In [11]:
# model.summary()

### Testing

In [12]:
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.callbacks import ModelCheckpoint


model = load_model('model_weights_FP.h5')

# Evaluate the model on the test set
loss, accuracy = model.evaluate(FP_x_test_resized, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 878us/step - accuracy: 0.9707 - loss: 0.0880
Test Loss: 0.07181327790021896
Test Accuracy: 0.9771000146865845


### Rearrange Weights for our dense layer (Format in Python is not practical for hardware) :  currently model_weights.txt is already rearranged

In [13]:
import numpy as np
text_file = np.loadtxt("model_weights_FP.txt")
# print(text_file[188:3428])

array_b = np.zeros(3240)
array_a = text_file[188:3428]
print(array_a)

for i in range(10):
    for j in range(4):
        for k in range(9*9):
            array_b[i + k * 10 + (j * 9 * 9 * 10)] = array_a[(j * 10) + (k * 40) + i]

array_final = np.zeros(3438)

array_final[0:188] = text_file[0:188]
array_final[188:3428] = array_b
array_final[3428:3438] = text_file[3428:3438]


with open('model_weights_FP_rearrange.txt', 'w') as file:
    for layer_weights in array_final:
            file.write(str(layer_weights) + '\n')

print("Model weights saved to model_weights.txt")


np.savetxt("model_weights_FP_rearrange.txt", array_final, delimiter=',', fmt='%f')  


[-0.42765197  0.08138641  0.01048726 ... -0.08735301 -0.49302652
  0.24226427]
Model weights saved to model_weights.txt


# Conv1

In [4]:
# import numpy as np
# from keras.models import Sequential
# from keras.layers import Conv2D

# # Load weights and biases from a text file
# weights_and_biases = np.loadtxt("model_weights.txt")

# # Assume the weights shape as per the convolutional kernel and filters
# weights_shape = (3, 3, 1, 4)  # Kernel size (3, 3), 4 filters
# weights = weights_and_biases[0:36].reshape(weights_shape)
# biases = weights_and_biases[36:40]

# # Build the model
# model = Sequential()
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
# model.layers[0].set_weights([weights, biases])

# # Load the pixel values from a text file
# img_array = np.loadtxt('demo_image.txt')  

# # Reshape the array to match the input shape of the model
# img_array = img_array.reshape((1, 42, 42, 1))  # (batch_size, height, width, channels)

# # Make predictions using the model
# conv1_output = model.predict(img_array)

# # Print output shape of the first convolutional layer
# print("Output shape of the first convolutional layer:", conv1_output.shape)

# # Save the convolutional layer output
# with open('Python_prog_out_conv.txt', 'w') as file:
#     for c in range(conv1_output.shape[3]):
#         file.write("Channel %d:\n" % (c + 1))
#         for i in range(conv1_output.shape[1]):
#             for j in range(conv1_output.shape[2]):
#                 file.write("%0.2f" % conv1_output[0, i, j, c])
#                 if j != conv1_output.shape[2] - 1:
#                     file.write(", ")
#             file.write("\n")
#         file.write("\n")

# print("Convolutional layer output saved")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Output shape of the first convolutional layer: (1, 40, 40, 4)
Convolutional layer output saved


c:\Users\Rishit Thakkar\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


# Max1

In [5]:
# # Load the weights and biases
# weights_and_biases = np.loadtxt("model_weights.txt")
# weights_shape = (3, 3, 1, 4)  # Kernel size of (3, 3) and 4 filters
# weights = weights_and_biases[0:36].reshape(weights_shape)  # Weights
# biases = weights_and_biases[36:40]  # Biases

# # Build the model
# model = Sequential()
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.layers[0].set_weights([weights, biases])

# # Load pixel values from a text file
# img_array = np.loadtxt('demo_image.txt') 
# img_array = img_array.reshape((1, 42, 42, 1))  # Reshape for the model

# # Predict using the model
# output = model.predict(img_array)

# # Output the shape of the result after Max Pooling layer
# print("Output shape of the first Max Pool layer:", output.shape)

# # Save the output of the Max Pooling layer
# with open('Python_prog_out_Max1.txt', 'w') as file:
#     for c in range(output.shape[3]):
#         file.write("Channel %d:\n" % (c + 1))
#         for i in range(output.shape[1]):
#             for j in range(output.shape[2]):
#                 file.write("%0.2f" % output[0, i, j, c])
#                 if j != output.shape[2] - 1:
#                     file.write(", ")
#             file.write("\n")
#         file.write("\n")

# print("Max 1 layer output saved")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Output shape of the first Max Pool layer: (1, 20, 20, 4)
Max 1 layer output saved


# Conv 2

In [6]:
# weights_and_biases = np.loadtxt("model_weights.txt")

# weights_shape = (3, 3, 1, 4)  # Assuming kernel size of (3, 3) and 4 filters
# weights2_shape = (3, 3, 4, 4)

# weights = weights_and_biases[0:36].reshape(weights_shape)  # weights
# biases = weights_and_biases[36:40]  # biases

# weights2 = weights_and_biases[40:184].reshape(weights2_shape)
# biases2 = weights_and_biases[184:188]

# #Model Below
# model = Sequential()
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))

# model.layers[0].set_weights([weights, biases])
# model.layers[2].set_weights([weights2, biases2])



# # Load the pixel values from a text file
# img_array = np.loadtxt('demo_image.txt')  

# # Reshape the array to match the input shape of the model
# img_array = img_array.reshape((1, 42, 42, 1))  # (batch_size, height, width, channels)

# conv1_output = model.predict(img_array)

# print("Output shape of the Second convolutional layer:", conv1_output.shape)

# # Save the convolutional layer output
# with open('Python_prog_out_conv2.txt', 'w') as file:
#     for c in range(conv1_output.shape[3]):  
#         file.write("Channel %d:\n" % (c + 1))

#         for i in range(conv1_output.shape[1]):  
#             for j in range(conv1_output.shape[2]):  
#                 file.write("%0.2f" % conv1_output[0, i, j, c])
#                 if j != conv1_output.shape[2] - 1:
#                     file.write(", ")
#             file.write("\n")

#         file.write("\n")

# print("Convolutional layer output saved")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Output shape of the Second convolutional layer: (1, 18, 18, 4)
Convolutional layer output saved


# Max 2

In [7]:
# weights_and_biases = np.loadtxt("model_weights.txt")

# weights_shape = (3, 3, 1, 4)  # Assuming kernel size of (3, 3) and 4 filters
# weights2_shape = (3, 3, 4, 4)

# weights = weights_and_biases[0:36].reshape(weights_shape)  # weights
# biases = weights_and_biases[36:40]  # biases

# weights2 = weights_and_biases[40:184].reshape(weights2_shape)
# biases2 = weights_and_biases[184:188]

# #Model Below
# model = Sequential()
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size = (2, 2)))

# model.layers[0].set_weights([weights, biases])
# model.layers[2].set_weights([weights2, biases2])

# img_array = np.loadtxt('demo_image.txt')

# img_array = img_array.reshape((1, 42, 42, 1))  # Reshape to (batch_size, height, width, channels)

# conv1_output = model.predict(img_array)

# print("Output shape of the 2nd MaxPool layer:", conv1_output.shape)

# # Save the convolutional layer output
# with open('Python_prog_out_max2.txt', 'w') as file:
#     for c in range(conv1_output.shape[3]):  
#         file.write("Channel %d:\n" % (c + 1))

#         for i in range(conv1_output.shape[1]):  
#             for j in range(conv1_output.shape[2]):  
#                 file.write("%0.2f" % conv1_output[0, i, j, c])
#                 if j != conv1_output.shape[2] - 1:
#                     file.write(", ")
#             file.write("\n")

#         file.write("\n")

# print("Maxpool layer 2 output saved")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Output shape of the 2nd MaxPool layer: (1, 9, 9, 4)
Maxpool layer 2 output saved


# Flatten

In [8]:
# weights_and_biases = np.loadtxt("model_weights.txt")

# weights_shape = (3, 3, 1, 4)  # Assuming kernel size of (3, 3) and 4 filters
# weights2_shape = (3, 3, 4, 4)
# weights3_shape = (324, 10)

# weights = weights_and_biases[0:36].reshape(weights_shape)  # weights
# biases = weights_and_biases[36:40]  # biases

# weights2 = weights_and_biases[40:184].reshape(weights2_shape)
# biases2 = weights_and_biases[184:188]




# #Model Below
# model = Sequential()
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size = (2, 2)))
# model.add(Flatten())


# model.layers[0].set_weights([weights, biases])
# model.layers[2].set_weights([weights2, biases2])


# img_array = np.loadtxt('demo_image.txt')

# img_array = img_array.reshape((1, 42, 42, 1))  # Reshape to (batch_size, height, width, channels)

# conv1_output = model.predict(img_array)

# # Save the convolutional layer output
# print("Output shape of the Flatten layer:", conv1_output.shape)

# # Save the flatten layer output
# with open('Python_prog_out_Flatten.txt', 'w') as file:
#     for i in range(conv1_output.shape[1]):
#         file.write("Neuron %d: %0.2f\n" % (i + 1, conv1_output[0, i]))

# print("Flatten layer output saved")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Output shape of the Flatten layer: (1, 324)
Flatten layer output saved


# Flatten and Dense

In [9]:
# weights_and_biases = np.loadtxt("model_weights.txt")

# weights_shape = (3, 3, 1, 4)  # Assuming kernel size of (3, 3) and 4 filters
# weights2_shape = (3, 3, 4, 4)
# weights3_shape = (324, 10)

# weights = weights_and_biases[0:36].reshape(weights_shape)  # weights
# biases = weights_and_biases[36:40]  # biases

# weights2 = weights_and_biases[40:184].reshape(weights2_shape)
# biases2 = weights_and_biases[184:188]

# weights3 = weights_and_biases[188:3428].reshape(weights3_shape)
# biases3 = weights_and_biases[3428:3438]

# #Model Below
# model = Sequential()
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu', input_shape=(42, 42, 1)))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Conv2D(4, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size = (2, 2)))
# model.add(Flatten())
# model.add(Dense(10, activation='relu'))

# model.layers[0].set_weights([weights, biases])
# model.layers[2].set_weights([weights2, biases2])
# model.layers[5].set_weights([weights3, biases3])


# img_array = np.loadtxt('demo_image.txt')
# img_array = img_array.reshape((1, 42, 42, 1))  # Reshape to (batch_size, height, width, channels)

# conv1_output = model.predict(img_array)

# # Save the convolutional layer output
# print("Output shape of the Dense layer:", conv1_output.shape)

# # Save the dense layer output
# with open('Python_prog_out_Dense.txt', 'w') as file:
#     file.write("Channel 1: \n")
#     for i in range(conv1_output.shape[1]):
#         file.write("%0.2f\n" % (conv1_output[0, i]))

# print("Dense layer output saved")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Output shape of the Dense layer: (1, 10)
Dense layer output saved
